In [2]:
# enable automatic reloading of the notebook
%load_ext autoreload
%autoreload 2

# Test Weight Store

In [29]:
from src.utils.weight_store import WeightStore
from transformers import AutoTokenizer

In [30]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [31]:
ws = WeightStore.load("../results/weight_stores/weight_store.en.wmt16.pickle")

In [32]:
print("{0:<14} {1:<14} {2}".format("Token", "Token Count", "IDF"))
for key, value in ws.word_count.items():
    idf = ws.get_idf(key)
    if idf > 5:
        print("{0:<14} {1:<14} {2}".format(key, value, idf))

Token          Token Count    IDF
▁911           400            6.057340978916829
▁Call          68             7.684020697785436
▁Professor     50             7.9863015696583695
▁Ad            60             7.807253338209384
mit            157            6.855532169355729
▁Sho           72             7.627667761234304
▁Girl          68             7.684020697785436
friend         60             7.807253338209384
▁call          891            5.182347188140368
▁voice         374            6.057340978916829
▁slightly      218            6.529055472566195
▁shak          121            7.114106157649439
▁college       201            6.6098595049814906
▁professor     515            5.9093140169401
▁Shan          159            6.842953387148868
non            199            6.619809835834658
▁shot          652            5.450428476278342
▁girlfriend    120            7.122336656785954
▁officer       851            5.288763948945246
▁needed        450            5.806659862880017
▁house

In [34]:
ws.index2word

{0: '<s>',
 360: '▁In',
 120138: '▁911',
 26265: '▁Call',
 4: ',',
 43552: '▁Professor',
 3145: '▁Ad',
 4007: 'mit',
 7: 's',
 47: '▁to',
 17779: '▁Sho',
 31: 'o',
 1916: 'ting',
 45327: '▁Girl',
 84240: 'friend',
 2: '</s>',
 10: '▁a',
 11782: '▁call',
 1919: '▁his',
 54479: '▁voice',
 4734: '▁only',
 161549: '▁slightly',
 149850: '▁shak',
 53: 'y',
 40466: '▁college',
 16030: '▁professor',
 41115: '▁Shan',
 6431: 'non',
 122930: '▁Lamb',
 30745: '▁told',
 35206: '▁police',
 764: '▁he',
 1902: '▁had',
 51876: '▁shot',
 169713: '▁girlfriend',
 136: '▁and',
 93324: '▁officer',
 44841: '▁needed',
 2046: '▁get',
 645: '▁over',
 2363: '▁their',
 18276: '▁house',
 5: '.',
 7228: '▁made',
 6275: '▁point',
 5154: '▁say',
 44: '▁"',
 1177: 'we',
 126: 'et',
 10269: '▁dog',
 58: '"',
 509: '▁was',
 2685: '▁there',
 923: '▁ali',
 272: 've',
 31895: '▁probably',
 1257: '▁up',
 3509: 'set',
 2804: '▁said',
 70: '▁the',
 103494: '▁dead',
 46667: '▁woman',
 25: "'",
 14449: '▁family',
 5470: '▁conta

In [43]:
token_id = 14
print(ws.get_word(token_id), ws.get_idf(ws.get_word(token_id)))

i 4.01416156835053


In [63]:
tokens = tokenizer(["I am going to work", "This is ok"], padding=True, return_tensors="pt")

In [64]:
tokens

{'input_ids': tensor([[   0,   87,  444, 7730,   47, 4488,    2],
        [   0, 3293,   83, 3664,    2,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0]])}

In [65]:
import torch

In [66]:
ws.index2word[0]

'<s>'

In [81]:
a = torch.tensor([ws.get_idf(ws.get_word(id.item())) for id in tokens["input_ids"].flatten()]).reshape(tokens["input_ids"].shape) * tokens["attention_mask"]

In [83]:
a

tensor([[1.0000e-05, 2.6387e+00, 4.9418e+00, 4.6224e+00, 9.2508e-01, 4.2642e+00,
         1.0000e-05],
        [1.0000e-05, 4.3117e+00, 1.6839e+00, 1.1918e+01, 1.0000e-05, 0.0000e+00,
         0.0000e+00]])

In [84]:
a / a.sum(dim=1).view(-1, 1).repeat(1,tokens["attention_mask"].shape[1])

tensor([[5.7497e-07, 1.5172e-01, 2.8414e-01, 2.6577e-01, 5.3190e-02, 2.4518e-01,
         5.7497e-07],
        [5.5823e-07, 2.4069e-01, 9.3998e-02, 6.6531e-01, 5.5823e-07, 0.0000e+00,
         0.0000e+00]])

In [50]:
a.sum(dim=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)